# Tutorial sobre procesamiento de imágenes

En este tutorial vamos a ver las herramientas básicas para trabajar con imágenes digitales usando Python. Para esto emplearemos un mínimo de paquetes disponibles para el Procesamiento de Imágenes Digitales.

## 1. Algunas utilidades de IPython Notebook... solo algunas

### Del sitio oficial:

__"__The goal of IPython is to create a comprehensive environment for __interactive__ and __exploratory computing__. To support this goal, IPython has three main components:
* An __enhanced interactive Python shell__.
* A __decoupled two-process communication model__, which allows for multiple clients to connect to a computation kernel, most notably the web-based notebook.
* An architecture for __interactive parallel computing__.

All of IPython is open source (released under the revised BSD license).__"__

### Sobre IPython Notebook:  

__"__The notebook __extends__ the console-based approach to __interactive computing__ in a qualitatively new direction, providing a web-based application suitable for capturing the whole computation process: __developing__, __documenting__, and __executing code__, as well as __communicating the results__. The IPython notebook combines two components:

* __A web application__: a browser-based tool for interactive authoring of documents which combine explanatory text, mathematics, computations and their rich media output.

* __Notebook documents__: a representation of all content visible in the web application, including inputs and outputs of the computations, explanatory text, mathematics, images, and rich media representations of objects.__"__

### 1.1. Navegación
* Tipos de celdas
* Shift-Enter, Ctrl-Enter, h
* Comandos del sistema**!**
* Obtener ayuda**?**
* Magic functions (**%lsmagic**, **%alias**, **%loadpy**, **%save**, **%run** ...)

#### Para Python 2.7

In [ ]:
from __future__ import print_function

#### Continuando con el tutorial

In [ ]:
b = 'py'
l = !dir | grep $b
for i in l:
    print(i)

In [ ]:
import scipy as sp
sp?

In [ ]:
%lsmagic

In [ ]:
%run -d collatz.py

In [ ]:
# %load collatz.py
from __future__ import print_function


def collatz(n):
    print("Collatz sequence of {}:".format(n))
    while n != 1:
        if n % 2:
            tmp = 3 * n + 1
            print("n = 3 x {} + 1 = {}".format(n, tmp))
        else:
            tmp = n // 2
            print("n = {} / 2 = {}".format(n, tmp))
        n = tmp


collatz(7)


In [ ]:
%save collatz.py 8

### 1.2. Cargando lo "imprescindible"

In [ ]:
%pylab inline

Un ejemplo bien sencillo de lo que trae %pylab.

In [ ]:
with xkcd():
    fig = plt.figure()
    ax = fig.add_axes((0.1, 0.2, 0.8, 0.7))
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    plt.xticks([])
    plt.yticks([])
    ax.set_ylim([-30, 10])

    data = np.ones(100)
    data[70:] -= np.arange(30)

    plt.annotate(
        'THE DAY I DISCOVERED BAD REGGAETON',
        xy=(70, 1), arrowprops=dict(arrowstyle='->'), xytext=(15, -10))

    plt.plot(data)

    plt.xlabel('time')
    plt.ylabel('my taste for popular music')

## 2. Introducción a la Morfología Matemática

* Soille, Pierre; Morphological Image Analysis, Principles and Applications, 2nd Ed.
* <a href="http://www.mamba-image.org">Mamba</a>


## 3. Trabajando con imágenes

### 3.1. Algunos paquetes interesantes
* __numpy__
* __scipy__ (__ndimage__)
* __cython__
* skimage
* sklearn
* opencv (no está en Anaconda)

### 3.2. Cargando lo "necesario"

In [ ]:
from scipy.misc import imread, imsave  # mas sencillos que en plt

In [ ]:
imcolor = imread('fish.jpg')
imshow(imcolor)

In [ ]:
imgray = imread('fish.jpg', True)
imshow(imgray, cmap=cm.gray)

### 3.3. Características de una imagen... o de una matriz de numpy

In [ ]:
print("color")
print(imcolor.shape)
print(imcolor.dtype)
print(imcolor[0,0])  # pixel en i,j

print("\ngray scale")
print(imgray.shape)
print(imgray.dtype)
print(imgray[0,0]) 

In [ ]:
imgray = array(imgray, dtype=uint8)
print(imgray.dtype)
imshow(imgray, cmap=cm.gray)

In [ ]:
imtest = imcolor.copy()  # rgb
#immono = imtest[:,:,2]
imtest[:, :, 0] = 0
imtest[:, :, 1] = 0
imshow(imtest)

In [ ]:
imshow(imgray.T, cmap=cm.gray)  # transpuesta de una matriz (importante: en escala de grises)

Cómo arreglar lo que sucede a continuación? Por qué ocurre?

In [ ]:
imtest = imgray[200:800, 400:1400]
imtest += 50
imshow(imgray, cmap=cm.gray)

In [ ]:
imsave('greyfish.jpg', imgray) # salvar en escala de grises

### 3.4. Cargando lo "suficiente": Cython

### Del sitio oficial:  
"Cython is a **programming language** that makes **writing C extensions** for the Python language **as easy as Python itself**. It aims to become a **superset of the Python language** which gives it high-level, object-oriented, functional, and dynamic programming. Its main feature on top of these is support for **optional static type declarations** as part of the language. The source code gets translated into **optimized C/C++ code** and **compiled** as Python **extension modules**. This allows for both very fast program execution and **tight integration with external C libraries**, while keeping up the high programmer productivity for which the Python language is well known."

In [ ]:
imcolor = imread('fish.jpg')
imgray = array(imread('fish.jpg', True), dtype=uint8)  # escala de grises con valores enteros en [0,255]

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

ax[0].imshow(imcolor)
ax[0].axis('off')
ax[0].set_title('color')
ax[1].imshow(imgray, cmap='gray')
ax[1].axis('off')
ax[1].set_title('gray')

## El problema  
Supongamos que no contamos con la implementanión del filtro __gradiente morfológico__ $g_B(f) = \delta_B(f) - \varepsilon_B(f)$ para un elemento estructurante en forma de cruz. Vamos a desarrollar un filtro que nos devuelva el gradiente morfológico de una imagen, y compararemos el tiempo de ejecución de este con los de algunas implementaciones existentes.

In [ ]:
from scipy.ndimage import morphology as morph

In [ ]:
se = zeros((3,3), dtype=uint8)  # elemento estructurante cruz
se[1,:] = se[:,1] = 1
se

In [ ]:
#%%timeit
grad = morph.morphological_gradient(imgray, structure=se)

In [ ]:
imshow(grad, cmap=cm.gray)

In [ ]:
#%%timeit
dil = morph.grey_dilation(imgray, structure=se)
ero = morph.grey_erosion(imgray, structure=se)
grad = dil - ero

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16, 8))

ax[0].imshow(dil, cmap=cm.gray)
ax[0].axis('off')
ax[0].set_title('dilation')
ax[1].imshow(ero, cmap=cm.gray)
ax[1].axis('off')
ax[1].set_title('erosion')
ax[2].imshow(grad, cmap=cm.gray)
ax[2].axis('off')
ax[2].set_title('gradient')

### Definiendo una función gradiente morfológico

In [ ]:
def gradient_1(im):
    h, w = im.shape
    grad = zeros(im.shape, dtype=uint8)
    for i in range(h):
        for j in range(w):
            pxmax = im[i, j]
            pxmax = max(pxmax, im[i-1, j] if i - 1 > 0 else 0)
            pxmax = max(pxmax, im[i, j+1] if j + 1 < w else 0)
            pxmax = max(pxmax, im[i+1, j] if i + 1 < h else 0)
            pxmax = max(pxmax, im[i, j-1] if j - 1 > 0 else 0)
            
            pxmin = im[i, j]
            pxmin = min(pxmin, im[i-1, j] if i - 1 > 0 else 255)
            pxmin = min(pxmin, im[i, j+1] if j + 1 < w else 255)
            pxmin = min(pxmin, im[i+1, j] if i + 1 < h else 255)
            pxmin = min(pxmin, im[i, j-1] if j - 1 > 0 else 255)
            grad[i, j] = pxmax - pxmin
    return grad

In [ ]:
%%timeit
grad = gradient_1(imgray)

In [ ]:
imshow(grad, cmap=cm.gray)

### Acceso rápido a los arrays de numpy

In [ ]:
def gradient_2(im):
    h, w = im.shape
    grad = zeros(im.shape, dtype=uint8)
    for i in range(h):
        for j in range(w):
            pxmax = im.item(i, j)
            pxmax = max(pxmax, im.item(i-1, j) if i - 1 > 0 else 0)
            pxmax = max(pxmax, im.item(i, j+1) if j + 1 < w else 0)
            pxmax = max(pxmax, im.item(i+1, j) if i + 1 < h else 0)
            pxmax = max(pxmax, im.item(i, j-1) if j - 1 > 0 else 0)
            
            pxmin = im.item(i, j)
            pxmin = min(pxmin, im.item(i-1, j) if i - 1 > 0 else 255)
            pxmin = min(pxmin, im.item(i, j+1) if j + 1 < w else 255)
            pxmin = min(pxmin, im.item(i+1, j) if i + 1 < h else 255)
            pxmin = min(pxmin, im.item(i, j-1) if j - 1 > 0 else 255)
            grad.itemset(i, j, pxmax - pxmin)
    return grad

In [ ]:
%%timeit
#%%prun
grad = gradient_2(imgray)

In [ ]:
imshow(grad, cmap=cm.gray)

### Cargando cythonmagic

In [ ]:
%load_ext Cython

In [ ]:
%%cython
import numpy as np
from itertools import product

cdef unsigned char maxval(unsigned char a, unsigned char b):
    return a if a > b else b

cdef unsigned char minval(unsigned char a, unsigned char b):
    return a if a < b else b

def gradient_3(im):
    #verificar que im sea de tipo uint8 de un solo canal
    cdef int i, j, h, w
    cdef unsigned char pxmax, pxmin
    h, w = im.shape
    grad = np.zeros(im.shape, dtype=np.uint8)
    for i, j in product(range(h), range(w)):
        pxmax = im.item(i, j)
        pxmax = maxval(pxmax, im.item(i-1, j) if i - 1 > 0 else 0)
        pxmax = maxval(pxmax, im.item(i, j+1) if j + 1 < w else 0)
        pxmax = maxval(pxmax, im.item(i+1, j) if i + 1 < h else 0)
        pxmax = maxval(pxmax, im.item(i, j-1) if j - 1 > 0 else 0)
        
        pxmin = im.item(i, j)
        pxmin = minval(pxmin, im.item(i-1, j) if i - 1 > 0 else 255)
        pxmin = minval(pxmin, im.item(i, j+1) if j + 1 < w else 255)
        pxmin = minval(pxmin, im.item(i+1, j) if i + 1 < h else 255)
        pxmin = minval(pxmin, im.item(i, j-1) if j - 1 > 0 else 255)
        grad.itemset(i, j, pxmax - pxmin)
    return grad

In [ ]:
%%timeit
grad = gradient_3(imgray)

In [ ]:
imshow(grad, cmap=cm.gray)

### Luego de un estudio de Cython y su integración con NumPy...

In [ ]:
%%cython

import numpy as np
cimport numpy as np

UINT8_DTYPE = np.uint8
ctypedef np.uint8_t UINT8_DTYPE_t

cdef inline UINT8_DTYPE_t maxval(UINT8_DTYPE_t a, UINT8_DTYPE_t b): return a if a >= b else b
cdef inline UINT8_DTYPE_t minval(UINT8_DTYPE_t a, UINT8_DTYPE_t b): return a if a <= b else b

def gradient_fast(np.ndarray[UINT8_DTYPE_t, ndim=2] im):
    cdef int i, j, h, w
    cdef UINT8_DTYPE_t pxmax, pxmin
    h = im.shape[0]
    w = im.shape[1]
    cdef np.ndarray[UINT8_DTYPE_t, ndim=2] grad = np.zeros((h, w), dtype=UINT8_DTYPE)
    
    for i in range(h):
        for j in range(w):
            pxmax = im[i, j]
            pxmax = maxval(pxmax, im[i-1, j] if i - 1 > 0 else pxmax)
            pxmax = maxval(pxmax, im[i, j+1] if j + 1 < w else pxmax)
            pxmax = maxval(pxmax, im[i+1, j] if i + 1 < h else pxmax)
            pxmax = maxval(pxmax, im[i, j-1] if j - 1 > 0 else pxmax)
            
            pxmin = im[i, j]
            pxmin = minval(pxmin, im[i-1, j] if i - 1 > 0 else pxmin)
            pxmin = minval(pxmin, im[i, j+1] if j + 1 < w else pxmin)
            pxmin = minval(pxmin, im[i+1, j] if i + 1 < h else pxmin)
            pxmin = minval(pxmin, im[i, j-1] if j - 1 > 0 else pxmin)
            grad[i, j] = pxmax - pxmin
    return grad

In [ ]:
%%timeit
grad = gradient_fast(imgray)

In [ ]:
imshow(grad, cmap=cm.gray)

### Detalles... los "*if*" tienen su costo también

In [ ]:
%%cython

import cython
import numpy as np
cimport numpy as np

UINT8_DTYPE = np.uint8
ctypedef np.uint8_t UINT8_DTYPE_t

cdef inline UINT8_DTYPE_t maxval(UINT8_DTYPE_t a, UINT8_DTYPE_t b): return a if a >= b else b
cdef inline UINT8_DTYPE_t minval(UINT8_DTYPE_t a, UINT8_DTYPE_t b): return a if a <= b else b

def gradient_faster(np.ndarray[UINT8_DTYPE_t, ndim=2] im):
    cdef:
        int i, j, h, w
        UINT8_DTYPE_t pxmax, pxmin

    h = im.shape[0]
    w = im.shape[1]
    cdef np.ndarray[UINT8_DTYPE_t, ndim=2] grad = np.zeros((h + 2, w + 2), dtype=UINT8_DTYPE)
    
    for i in range(1, h - 1):
        for j in range(1, w - 1):
            pxmax = im[i, j]
            pxmax = maxval(pxmax, im[i-1, j])
            pxmax = maxval(pxmax, im[i, j+1])
            pxmax = maxval(pxmax, im[i+1, j])
            pxmax = maxval(pxmax, im[i, j-1])
            
            pxmin = im[i, j]
            pxmin = minval(pxmin, im[i-1, j])
            pxmin = minval(pxmin, im[i, j+1])
            pxmin = minval(pxmin, im[i+1, j])
            pxmin = minval(pxmin, im[i, j-1])
            grad[i, j] = pxmax - pxmin
    return grad[1:-1, 1:-1]

In [ ]:
%%timeit
grad = gradient_faster(imgray)

In [ ]:
imshow(grad, cmap=cm.gray)

### Listo para usar?
Simplemente salvamos el código en un fichero *.pyx*. Luego se crea un fichero *setup.py* y lo compilamos con el comando *python setup.py build_ext --inplace*. Esto creará un módulo .so (en linux) listo para importar desde código *Python*. El código de *setup.py* es muy sencillo:  
<code>
from distutils.core import setup  
from Cython.Build import cythonize  
  
setup(ext_modules = cythonize("gradient.pyx"))
</code>

In [ ]:
%save -r gradient 86

In [ ]:
!mv gradient.ipy gradient.pyx

In [ ]:
!gedit gradient.pyx  # para quitar %%cython

In [ ]:
!gedit setup.py  # crear este fichero con lo dicho anteriormente

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
!ls  # para ver lo que se genera

In [ ]:
from gradient import gradient_faster as the_fastest  # probando resultados

In [ ]:
#%%timeit
imgrad = the_fastest(imgray)

In [ ]:
imshow(imgrad, cmap=cm.gray)